# Using Linear Algebra to compute Economic Ouput and Greenhouse Gases Impact

In [2]:
import pandas as pd
import numpy as np

path = 'bb21a10summarytables.xlsx'
# Input output table
df = pd.read_excel(path,
    sheet_name = 22,
    usecols = "C:L",
    header = None,
    skiprows = 52,
    nrows = 10)
z = np.array(df,dtype = float) # £ million

# Output por sector
dfx = pd.read_excel(path,
    sheet_name = 22,
    usecols = "C:L",
    header = None,
    skiprows = 75,
    nrows = 1)
x = np.array(dfx, dtype = float)[0] # £ million

# Intensidade de GEE
pathe = 'ghgintensity_uk.xlsx'
dfem = pd.read_excel(pathe,
    sheet_name = 0,
    usecols = "B:X",
    skiprows = 1,
    header = None,
    nrows = 10)
E = np.array(dfem, dtype = float) #Thousand tonnes CO2 /£ million

C:\Users\Tiago Vieira da Silv\AppData\Local\Programs\Python\Python310\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


## Excercise a)

To calculate the technical coefficient matrix, the inverse of the total output vector is multiplied by an identity matrix, which is then multiplied by the intermediates table. 

In [3]:
def coef_mat(z, x):
    coef = np.dot(z, np.linalg.inv(x * np.identity(len(x))))
    return coef

## Exercise b)

We can then apply the Leontief inverse to obtain the technology matrix — matrix (B), which consists of taking the inverse of an identity matrix minus the technical coefficient matrix ([I — A]^-1).

We then sum each column to find out the impact of each sector on GDP.

In [4]:
def leon(a, dif = 1e-6):
    return np.linalg.inv(np.identity(len(a)) - a)

def impact_output(a):
    return leon(a).sum(axis = 0)

def maximo(a):
    sector = 1 + np.argmax(impact_output(a))
    return "Sector " + str(sector)    

## Excercise c)

We can use the Leontief Inverse matrix to calculate de Greenhouse Gases impact doing the dot sector between the Leontief and a vector with the tons of CO2 emissions per million £.

In [5]:
def impact_ghg(a, ghg):
    return np.dot(ghg, leon(a))/1000

## Exercise d)

I calculate wellbeing that is caused by each sector using the difference between the economic impact output and greenhouse gases impact.

In [6]:
def wellbeing(a, ghg, p = 50):
    return impact_output(a)-impact_ghg(a, ghg)*p   

def maxwell(a, ghg, p = 50):
    return "Sector " + str(1 + np.argmax(wellbeing(a, ghg, p)))

def price_range(a, ghg, p = 50):
    default_sector = maxwell(a, ghg, p)
    max = p
    min = p
    for price in range(p, 201):
        if (maxwell(a, ghg, price) == default_sector):
            max = price
    for price in range(p, -1, -1):
        if (maxwell(a, ghg, price) == default_sector):
            min = price
    return  [min, max]   

### Computing with 2019 values

In [8]:
# Vector with all GHG in 2019 
ghg_2019 = E[:,22] #Thousand tonnes CO2 /£ million
a = coef_mat(z, x)

print("Impact Output:")
print(impact_output(a))
print("\nSector with the biggest impact: ")
print(maximo(a))
print("\nSector that causes the most wellbeing: ")
print(maxwell(a, ghg_2019))
print("\nInterval in which the emissions price can change and Sector remains the same: ")
print(price_range(a, ghg_2019))

Impact Output:
[2.42476711 2.5312143  2.49521541 2.0024569  1.87926231 2.03488466
 1.45453925 1.85774875 1.78260567 1.67858368]

Sector with the biggest impact: 
Sector 2

Sector that causes the most wellbeing: 
Sector 3

Interval in which the emissions price can change and Sector remains the same: 
[5, 161]
